In [ ]:
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.model_selection import train_test_split

from tqdm import tqdm
import numpy as np
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import TensorDataset
import torch.optim as optim
import torch.utils.data as data
from torchvision import datasets
import torchvision.transforms as transforms

device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # use gpu

In [ ]:
# loading data
usc_raw_data = pd.read_csv('/content/drive/MyDrive/Industrial Team Project/USCHAD_rawdata.csv')

In [ ]:
# only features and label.
usc_unprocessed = usc_raw_data[['acc_x','acc_y','acc_z','gyro_x','gyro_y','gyro_z','label']]

In [ ]:
# window_size: size of time window
# step: overlapping
# data: dataset
def time_windows(window_size,overlapping,data):
  sigmentation_data_temp = []
  sigmentation_data = []
  sigmentation_label = []
  for i in range(0,len(data),overlapping):
    acc_x = data['acc_x'].values[i:i+window_size]
    acc_y = data['acc_y'].values[i:i+window_size]
    acc_z = data['acc_z'].values[i:i+window_size]
    gyro_x = data['gyro_x'].values[i:i+window_size]
    gyro_y = data['gyro_y'].values[i:i+window_size]
    gyro_z = data['gyro_z'].values[i:i+window_size]
    total_label = data['label'].values[i:i+window_size]
    label = Counter(total_label).most_common()[0][0]
    sigmentation_data_temp.append([acc_x,acc_y,acc_z,gyro_x,gyro_y,gyro_z])
    sigmentation_arr = np.asarray(sigmentation_data_temp)
    sig_size = sigmentation_arr.shape
    if sig_size[2] == window_size:
      sigmentation_arr.reshape(window_size,6)
      sigmentation_data.append(sigmentation_arr)
      sigmentation_label.append(label)
      sigmentation_data_temp = []
    else:
      sigmentation_data_temp = []

  sigmentation_data_arr = np.asarray(sigmentation_data)
  sigmentation_label_arr = np.asarray(sigmentation_label)

  return sigmentation_data_arr,sigmentation_label_arr

In [ ]:
uci_readings_75_overlapping,uci_labels_75_overlapping = time_windows(128,32,usc_unprocessed)

In [ ]:
uci_labels_75_overlapping = uci_labels_75_overlapping - 1

In [ ]:
x_train_75_overlapping,x_val_test_75_overlapping,y_train_75_overlapping,y_val_test_75_overlapping = train_test_split(uci_readings_75_overlapping,uci_labels_75_overlapping,test_size = 0.3)
x_val_75_overlapping,x_test_75_overlapping,y_val_75_overlapping,y_test_75_overlapping = train_test_split(x_val_test_75_overlapping,y_val_test_75_overlapping,test_size = 0.3 )

In [ ]:
train_dataset_75_overlapping = TensorDataset(torch.from_numpy(x_train_75_overlapping).to(torch.float32),torch.from_numpy(y_train_75_overlapping).to(torch.float32))
val_dataset_75_overlapping = TensorDataset(torch.from_numpy(x_val_75_overlapping).to(torch.float32),torch.from_numpy(y_val_75_overlapping).to(torch.float32))
test_dataset_75_overlapping = TensorDataset(torch.from_numpy(x_test_75_overlapping).to(torch.float32),torch.from_numpy(y_test_75_overlapping).to(torch.float32))

In [ ]:
train_loader_75_overlapping = data.DataLoader(dataset = train_dataset_75_overlapping,batch_size = 64,shuffle = True)
val_loader_75_overlapping = data.DataLoader(dataset = val_dataset_75_overlapping,batch_size = 64,shuffle = True)
test_loader_75_overlapping = data.DataLoader(dataset = test_dataset_75_overlapping,batch_size = 64,shuffle = False)

In [ ]:
 # now define a class of CNN model
class CNN_75_overlapping(nn.Module):
  def __init__(self):
    super(CNN_75_overlapping, self).__init__()
    self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.MaxPool2d(kernel_size=2, stride=2))
    self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.MaxPool2d(2,2))
    self.layer3 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2),
            nn.ReLU()
    )
    # self.drop_out = nn.Dropout()
    self.fc1 = nn.Linear(3072, 512)
    self.fc3 = nn.Linear(512,12)
  def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.reshape(out.size(0), -1)
        # out = self.drop_out(out)
        out = self.fc1(out)
        out = self.fc3(out)
        return out
        
model_75_overlapping = CNN_75_overlapping().to(device)

In [ ]:
epoches = 50
lr = 0.001
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model_75_overlapping.parameters(),lr=lr, weight_decay=0.0001,momentum = 0.8)

In [ ]:
# training
loss_list = []
for epoch in range(epoches):
    acc = 0
    correct = 0
    total = 0
    loss_num = 0
    runtime_loss = 0
    for (imgs,labels) in train_loader_75_overlapping:
        imgs = imgs.reshape(len(imgs),1,32,24)     
        imgs = imgs.to(device)
        labels = labels.type(torch.LongTensor)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model_75_overlapping(imgs)
        loss = criterion(outputs, labels)
        runtime_loss += loss.item()
        loss.backward()
        optimizer.step()
        total += labels.size(0)
        _,predicted = torch.max(outputs.data,1)
        correct += (predicted == labels).sum().item()
        loss_num+=1
    acc = 100 * correct / total
    epoch_loss = runtime_loss / loss_num
    loss_list.append(epoch_loss)
    loss_num = 0
    print("Epoch: ",epoch+1, "accuracy: ", acc,' %',"Loss: ",epoch_loss)

Epoch:  1 accuracy:  55.79658258746949  % Loss:  1.2431095207767109
Epoch:  2 accuracy:  74.17900732302685  % Loss:  0.7280475162318544
Epoch:  3 accuracy:  80.69812855980472  % Loss:  0.5288743708092762
Epoch:  4 accuracy:  82.41334418226201  % Loss:  0.467324533382514
Epoch:  5 accuracy:  83.6013018714402  % Loss:  0.43328468993104585
Epoch:  6 accuracy:  84.3840520748576  % Loss:  0.40938001567193544
Epoch:  7 accuracy:  84.84621643612694  % Loss:  0.39162165780903524
Epoch:  8 accuracy:  85.28722538649308  % Loss:  0.3754146368427902
Epoch:  9 accuracy:  85.74613506916192  % Loss:  0.3631568354914017
Epoch:  10 accuracy:  86.15785191212368  % Loss:  0.34920329129968797
Epoch:  11 accuracy:  86.44426362896664  % Loss:  0.33759725092290965
Epoch:  12 accuracy:  87.08218063466232  % Loss:  0.32711482725642105
Epoch:  13 accuracy:  86.97965825874695  % Loss:  0.31984746131297825
Epoch:  14 accuracy:  87.35394629780309  % Loss:  0.309342167364447
Epoch:  15 accuracy:  87.46786004882019 

In [ ]:
# test
correct_test = 0

total_test = 0

with torch.no_grad():

  for (imgs_test,labels_test) in test_loader_75_overlapping:

    imgs_test = imgs_test.reshape(len(imgs_test),1,32,24)

    imgs_test = imgs_test.to(device)

    labels_test = labels_test.type(torch.LongTensor)

    labels_test = labels_test.to(device)

    outputs_test = model_75_overlapping(imgs_test)

    labels_test = labels_test.reshape(labels_test.shape[0],)

    predicted_test = torch.max(outputs_test.data,1)[1]
      
    total_test += labels_test.size(0)
      
    correct_test += (predicted_test == labels_test).sum().item()

    test_accuracy = 100*correct_test/total_test

print('Test accuracy is: ',test_accuracy)

Test accuracy is:  87.14249557074159
